# Im_reduc example

This is a basic image reduction script based on the data processing techniques taught in the physics 2070 course and York university that were previously carried through with IRAF. This script aims to be the first of many demonstrations of a more automatic system that still retains full user control and more education value than just clicking a button. 

## Imports

The first aspect of every script is importing the external code that the script will 'call to'. These are known as import statements. First we import the main Draco file and then we import the Table and fits libraries from astropy. Next we import the two most fundamental libraries for anything science or computing related in python, numpy and matplotlib.

In [ ]:
import draco
from astropy.io import fits
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt

from astropy.nddata import CCDData
from ccdproc import Combiner
import ccdproc
import astropy.units as u
from astropy.utils.misc import isiterable


## Declaring file prefixes and locations

Here are the standard file prefixes for this reduction script. Data was taken in 8 band filters in December of 2018 of comet 46P Wirtanen (4 filters are clear). Along with 'lights', 'flats' and 'bias' images were taken for reduction.

In [ ]:
file_path = 'C:/Users/mucep/Offline/Draco-test/'
im_prefix = 'Draco-'
c46P_prefix = '46P_'
BIAS_suffix = '.BIAS'
FLAT_prefix = 'FLAT_'
FLAT_suffix = '.FLAT'
filters = ['BLUE', 'CLEAR', 'EIGHT', 'GREEN', 'LUMINANCE', 'RED', 'SEVEN', 'SIX']


## Making the master BIAS

Making a master bias image in DRACO is as easy as one line. By feeding the `draco.mast_reduc` function the folder location of the BIAS images, the filename prefix (text before the index numbers), the filename suffix (text after the index numbers), the number of BIAS images taken, and a Boolean value of whether or not to edit out cosmic rays from the BIAS images before stacking (takes extra time to compute), draco will return the master BIAS image which we store as `master_bias`.

By uncommenting the second line will write the master BIAS image to a fits file in the scripts home folder. Future versions of DRACO will better handle file input/output.


In [ ]:
master_bias = draco.mast_reduc(file_path + 'Bias/', im_prefix, BIAS_suffix, 10, True)
#fits.writeto('master_bias.fits', master_bias)


## Setting up the FLATS in memory



In [ ]:
reduc_filters =  ['BLUE', 'CLEAR', 'LUMINANCE', 'SEVEN', 'SIX']
size = master_bias.shape
im_count = 5
master_i_Flat = []


## Reading and reducing the FLATS into Master FLATS



In [ ]:
for i in range(len(reduc_filters)):
    print('Starting ', reduc_filters[i], ' filter...')
    flat_i = draco.mast_flat(file_path + 'Flats/' + reduc_filters[i] + '/', im_prefix + reduc_filters[i] + '-', FLAT_suffix, 10, False, master_bias)
    #flat_i = draco.imarith(flat_i, '-', master_bias)
    #flat_i = draco.norm_flat(np.array(flat_i.data))
    master_i_Flat.append(flat_i)
    #fits.writeto('master_' + reduc_filters[i] + '_Flat.fits', master_i_Flat[i])


## Reading and reducing the LIGHTS into Fully Reduced LIGHTS



In [ ]:
light_n = []
series_all = []
for n in  range(len(reduc_filters)):
    series_n = draco.get_series(file_path + 'Lights/', c46P_prefix + reduc_filters[n] + '-', '', 5, False)
    series_n = draco.series_arith(series_n, '-', master_bias, 5)
    series_n = draco.series_arith(series_n, '/', master_i_Flat[n], 5)
    sky = draco.sky_est(series_n)
    series_n = draco.series_arith(series_n, '-', sky, 5)
    series_all.append(series_n)
    combine_list = []
    for t in range(5):
        combine_list.append(CCDData(series_n[:, :, t], unit=u.adu))
    combine = Combiner(combine_list)
    reduced_filter_nocr = combine.average_combine()
    reduced_filter = ccdproc.cosmicray_lacosmic(reduced_filter_nocr)
    light_n.append(reduced_filter)
    print(reduc_filters[n], ' Filter lights complete.')

## Plotting a sample image



In [ ]:
series_all = np.array(series_all)
print(series_all.shape)
plt_series = series_all[3, :, :, 1]
draco.plt_fits(light_n[3], 'viridis')
draco.plt_fits(plt_series, 'viridis')
draco.plt_flat(master_i_Flat[3], 'viridis')

## Check that DRACO and IRAF agree

It is important to make sure that DRACO and IRAF both give the same result. This first test will check whether the BIAS images constructed are the same, and whether the master FLATS images are the same.

In [ ]:
master_bias_IRAF = fits.getdata('MasterBias_IRAF.fits')

ag = np.abs(master_bias_IRAF - master_bias)
print(np.mean(ag))
draco.plt_fits(ag, 'viridis')

In [ ]:
master_Bflat_IRAF = fits.getdata('MasterLuminanceFlat_IRAF.fits')

ag2 = np.abs((master_Bflat_IRAF/np.mean(master_Bflat_IRAF)) - master_i_Flat[3])
print(np.mean(ag2))
print(np.mean(master_Bflat_IRAF))
print(np.mean(master_i_Flat))
draco.plt_fits(ag2, 'viridis')
# fits.writeto('cool_flat_diff.fits', ag2)